**GPU로 설정하셔야 모델을 정상적으로 실행할 수 있습니다.**

In [ ]:
#@title Install dependencies
# install dependencies: (use cu101 because colab has CUDA 10.1)
#!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1

!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab


ERROR: Operation cancelled by user
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8sph6ih3
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8sph6ih3
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done


In [ ]:
#@title Install detectron2
%cd /content/
# install detectron2:
!git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2
!pip install -r requirements.txt
!python setup.py install
!pip install git+https://github.com/facebookresearch/fvcore.git


In [ ]:
#@title Install detectron2-ResNeSt
%cd /content/
# install detectron resnest
!python -m pip install 'git+https://github.com/zhanghang1989/detectron2-ResNeSt.git'

In [ ]:
### 밑에 코드 전에 반드시 런타임을 다시 재시작 및 모두 실행하셔야 에러가 나지 않습니다!!!

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect

# import some common libraries
import numpy as np
import pandas as pd
import os
import sys
import json
import cv2
import random
import joblib
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm

font_location = "/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/Font/NanumGothic.ttf" # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = torch.device('cpu')

%matplotlib inline

Using CUDA


In [ ]:
backbone_path="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/backbonefolder"
original_path="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023"

In [ ]:
# 저희들의 validation image경로에 맞게 아래의 get_set_number함수를 새로 제작했습니다!
# set1, set2, ... set10까지 return합니다.

listofproducts = []
def get_set_number(filename):
    if filename.startswith('cart_'):
      count = str(int(filename.split('.')[0].split('_')[1][:2]))
      num = f'set{count}'
      return num
def get_dicts(img_dir, jsonfilename):
    listofproducts.clear()
    json_file = os.path.join(jsonfilename)
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        new_name = get_set_number(v["filename"])

        filename = os.path.join(os.path.join(os.path.join(img_dir, "validation"), new_name), v["filename"])
        print(filename)
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = v["regions"]
        objs = []
        for i, anno in enumerate(annos):
            #assert not anno["region_attributes"]
            value = anno["region_attributes"]['상품']
            listofproducts.append(value)
            list(set(listofproducts))
            anno_val = anno["shape_attributes"]
            px = anno_val["all_points_x"]
            py = anno_val["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts



In [ ]:
# json파일 중 현재 set7은 진행되지 않은 상태로 이해하고 코드 작성했습니다!

json_path1_5='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation1_5.json'
json_path6_8_9='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation_set6_9.json'
json_path7='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation_set7.json'
json_path10='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation_set10.json'

dataset_dict_result = []
jsonlist = [json_path1_5, json_path6_8_9, json_path7, json_path10]

for jsons in jsonlist:
    if not os.path.exists(jsons):
        None
    else:
      dict_res = get_dicts(original_path, jsons)
      dataset_dict_result.append(dict_res)

/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set1/cart_0101.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set1/cart_0102.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set1/cart_0103.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set1/cart_0104.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set2/cart_0201.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set2/cart_0202.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set2/cart_0203.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set2/cart_0204.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set3/cart_0301.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set3/cart_0302.jpg
/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation/set3/cart_0303.jpg
/content/d

In [ ]:
d = 'validation'
DatasetCatalog.clear()
DatasetCatalog.register("validation_" + d, lambda d=d: get_dicts(original_path))
MetadataCatalog.get("validation_" + d).set()
MetadataCatalog.get("validation_" + d).set(thing_classes=["box"])
cj_metadata = MetadataCatalog.get("validation_" + d)

In [ ]:
# 해당 코드는 전체 이미지를 출력하는 코드입니다!
# 현재는 모든 라벨링을 이미지에 그릴 때 위에서 폰트를 설정해도 한글폰트가 깨지는데 이는 찾아보고 해결해보겠습니다!
for i, d in enumerate(dataset_dict_result):
    for j in range(len(d)):
        img = cv2.imread(d[j]["file_name"])
        visualizer = Visualizer(img[:, :, ::-1], metadata=cj_metadata, scale=0.5)
        out = visualizer.draw_dataset_dict(d[j])
        cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
df = pd.DataFrame(columns=['BBOX',])
cnt = 0
for i, d in enumerate(dataset_dict_result):
    for j in range(len(d)):
        cnt += 1
        box = []
        for k in range(len(d[j]['annotations'])):
            box.append(d[j]['annotations'][k]['bbox'])
        df.loc[cnt] = [box]
df

,BBOX
1,"[[551, 1259, 692, 1361], [129, 1173, 432, 1388..."
2,"[[154, 1069, 266, 1390], [662, 1171, 890, 1329..."
3,"[[665, 932, 878, 1312], [520, 1105, 695, 1354]..."
4,"[[221, 1143, 316, 1228], [638, 894, 854, 1048]..."
5,"[[238, 966, 478, 1126], [260, 1118, 488, 1256]..."
6,"[[291, 1112, 456, 1208], [654, 968, 953, 1139]..."
7,"[[501, 370, 660, 475], [509, 457, 662, 561], [..."
8,"[[24, 336, 323, 615], [620, 1010, 797, 1164], ..."
9,"[[703, 925, 804, 1019], [694, 724, 826, 925], ..."
10,"[[730, 408, 986, 666], [747, 667, 947, 806], [..."


In [ ]:
# 이 위치에 위에서 얻은 좌표값을 통해 각각의 이미지를 가지고 오는 코드를 추가해주시면 됩니다!!

In [ ]:
import cv2

# 이미지 로드 함수
def load_image(image_path):
    return cv2.imread(image_path)

# 이미지 저장 함수
def save_image(image, file_name):
    cv2.imwrite(file_name, image)

cnt = 0
imagefolderpath='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder'
for i, d in enumerate(dataset_dict_result):
    for j in range(len(d)):
        cnt += 1

        # BBOX에서 좌표를 가져옵니다.
        boxes = df.loc[cnt, 'BBOX']

        # 파일 이름을 가져옵니다. 이 부분이 dataset_dict_result의 구조에 따라 달라질 수 있습니다.
        image_path = d[j]['file_name']

        img = load_image(image_path)
        if img is None or img.size == 0:
            print(f"Image not loaded properly: {image_path}")
            continue

        for k, coordinates in enumerate(boxes):
            x_min, y_min, x_max, y_max = map(int, coordinates)

            # 좌표 범위가 유효한지 확인
            if x_min >= x_max or y_min >= y_max or x_max > img.shape[1] or y_max > img.shape[0]:
                print(f"Invalid coordinates for image {image_path}: {coordinates}")
                continue

            cropped_image = img[y_min:y_max, x_min:x_max]
            if cropped_image.size == 0:
                print(f"Cropped image is empty for coordinates: {coordinates}")
                continue
            #기존 코드에서는 root폴더에 저장되므로 폴더안에 넣는 코드로 변경하였습니다!
            #save_path = f'cropped_box_{cnt}_{k}.png'
            save_path = f'{imagefolderpath}/cropped_box_{cnt}_{k}.png'

            save_image(cropped_image, save_path)
            print(f"Saved {save_path}")



Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_0.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_1.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_2.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_3.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_4.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_5.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_6.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_7.png
Saved /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑PreprocessedFolder/cropped_box_1_8.png
Saved /content/driv

In [ ]:
#### 밑에서부터 학습하고 확인하는 코드

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=1f9f86bc6cf7fae6734b27b34672e8152dfe18f774d2409ff41f33aa78e4a342
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
wandb.init(project='test-detectron2-wandb-watch', sync_tensorboard=True)

wandb: Currently logged in as: pje123131. Use `wandb login --relogin` to force relogin


In [ ]:
# validation training json
json_path1_5='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation1_5.json'
json_path6_9='/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/validation_set6_9.json'

dataset_dict_result = []
jsonlist = [json_path1_5, json_path6_9]


In [ ]:
# 데이터 등록하는 예시 코드
#if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")


In [ ]:
from detectron2.data.datasets import register_coco_instances

# 훈련 데이터셋 경로
train_dataset_path = "/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/Validation TrainTestFolder/Train"

register_coco_instances("custom_train_dataset", {}, train_dataset_path, jsonlist)


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
train_dataset_path = "/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/Validation TrainTestFolder/Train"
import os
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (train_dataset_path,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 113  # only has one class (ballon)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
wandb.watch(trainer.model)
trainer.train()

[08/04 07:51:05 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

KeyError: ignored

In [ ]:
### 아래에 있던 기존의 코드는 detectron2 학습코드였지만 현재는 위에서 얻는 이미지와 좌표값을 통해 백본 모델을 돌리는 코드로 변경합니다!!
### 기존의 코드를 확인하시려면 지은님의 파일인 /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/detectron2_ipynb의_사본.ipynb에서 확인바랍니다!!

In [ ]:
sys.path.append(backbone_path)
from ModelClass import ResNet, GoogLeNet, ResGoogLeNet

In [ ]:
backbonemodel = [ResNet(), GoogLeNet(), ResGoogLeNet()]
if torch.cuda.is_available():
    ResNet().cuda()
    GoogLeNet().cuda()
    ResGoogLeNet().cuda()
def get_load_model(modelname):
    if modelname == ResNet():
      backbonemodelname = 'ResNet'
    elif modelname == GoogLeNet():
      backbonemodelname = 'GoogLeNet'
    else:
      backbonemodelname = 'ResGoogLeNet'
    path = os.path.join(backbone_path, f'backbonemodel-{backbonemodelname}.h5')
    return path
path = get_load_model(backbonemodel[0])
#path = get_load_model(backbonemodel[1])
#path = get_load_model(backbonemodel[2])
print(path)
backbonemodel[0] = joblib.load(path)
#backbonemodel[1] = joblib.load(path)
#backbonemodel[2] = joblib.load(path)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 104MB/s]
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 94.5MB/s]


/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/backbonefolder/backbonemodel-ResGoogLeNet.h5


In [ ]:
# 전체 라벨과 상응하는 상품명을 담은 코드입니다.
# 아래 코드에 필요하기에 작성되었습니다.
labeledimg_dir="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑"
labelnamelist = []

for labelname in set(os.listdir(labeledimg_dir)):
  labelnum = labelname.split(')')[0]
  label = {labelnum: labelname.split(') ')[1].split('.')[0]}
  labelnamelist.append(label)
labelnamelist = [dict(t) for t in {tuple(d.items()) for d in labelnamelist}]

In [ ]:
predict_list = []

for filename in os.listdir(imagefolderpath):
  path = os.path.join(imagefolderpath, filename)
  img = load_image(path)
  # 이미지 전처리
  img = cv2.resize(img, (224, 224))
  img = img.astype('float32') / 255.
  img = torch.from_numpy(img.transpose(2, 0, 1)).unsqueeze(0)

  # 모델 예측
  if torch.cuda.is_available():
      backbonemodel[0][0].cuda()
      img = img.cuda()
  else:
      backbonemodel[0][0] = torch.load(path, map_location=torch.device('cpu'))
  backbonemodel[0][0].eval()
  with torch.no_grad():
      output = backbonemodel[0][0](img)
      _, predicted = torch.max(output.data, 1)

  predict_list.append(predicted.item())
print(predict_list)

[20, 26, 26, 70, 70, 70, 70, 26, 70, 44, 5, 60, 70, 70, 47, 89, 67, 70, 70, 89, 89, 89, 81, 70, 59, 37, 16, 47, 70, 21, 70, 70, 70, 85, 5, 34, 5, 85, 21, 81, 34, 96, 21, 97, 21, 47, 44, 83, 21, 9, 9, 9, 44, 34, 70, 16, 26, 83, 70, 91, 91, 70, 44, 97, 47, 75, 62, 9, 9, 44, 85, 47, 81, 83, 20, 44, 44, 44, 44, 83, 44, 44, 44, 62, 47, 47, 47, 47, 5, 44, 62, 85, 44, 44, 34, 8, 44, 44, 5, 62, 5, 44, 62, 62, 85, 44, 62, 47, 52, 85, 47, 5, 62, 44, 44, 44, 70, 20, 83, 44, 34, 90, 47, 70, 85, 85, 52, 44, 47, 70, 47, 44, 34, 44, 44, 44, 62, 5, 44, 47, 47, 44, 44, 47, 44, 44, 44, 47, 85, 44, 9, 5, 5, 62, 62, 9, 44, 44, 62, 62, 52, 44, 44, 47, 62, 70, 70, 70, 81, 44, 34, 85, 44, 34, 89, 47, 84, 5, 5, 9, 85, 62, 70, 70, 44, 9, 34, 70, 47, 44, 70, 91, 44, 20, 44, 85, 20, 44, 5, 47, 62, 5, 81, 44, 5, 81, 44, 5, 44, 62, 5, 85, 5, 5, 62, 62, 44, 5, 5, 44, 44, 5, 44, 5, 5, 44, 5, 5, 44, 5, 5, 44, 44, 5, 5, 5, 5, 5, 42, 102, 42, 44, 70, 62, 47, 62, 44, 44, 70, 70, 62, 62, 62, 62, 70, 62, 5, 70, 83, 21, 90

In [ ]:
from collections import Counter
df = pd.DataFrame(columns=['Index','Cnt'])
prev_index = '1'
count = 0
box = []
for filename in os.listdir(imagefolderpath):
  prevtmp_index = str(filename.split('_')[2])
  nexttmp_index = str(filename.split('_')[3])

  if prev_index != prevtmp_index:
    df.loc[int(prev_index)] = [prev_index, Counter(box)]
    prev_index = prevtmp_index
    box.clear()
  for ln in labelnamelist:
    if str(predict_list[count]) in ln:
      box.append(str(predict_list[count]) + '-' + ln[str(predict_list[count])])
  count += 1
df.loc[int(prevtmp_index)] = [prevtmp_index, Counter(box)]
box.clear()

In [ ]:
df

,Index,Cnt
1,1,"{'20-탐스제로레몬': 1, '26-볶음 간짬뽕 ..."
2,2,"{'60-맥콜 제로': 1, '70-롯데 수박바 에이드..."
3,3,"{'37-알로에베라킹 프리미엄 제로': 1, '16-피..."
4,4,"{'34-프레첼 갈릭버터맛': 2, '85-월매 쌀ᄆ..."
5,5,"{'44-게토레이블루': 2, '34-프레첼 갈릭버터..."
6,6,"{'62-비락 식혜 pet': 1, '44-게토레이블루': ..."
7,7,"{'44-게토레이블루': 13, '62-비락 식혜 pet':..."
8,8,"{'70-롯데 수박바 에이드': 2, '20-탐스제로레..."
9,9,"{'44-게토레이블루': 16, '47-코카콜라제로': 7..."
10,10,"{'70-롯데 수박바 에이드': 3, '81-오뚜기 콕ᄏ..."
